In [39]:
!git clone https://github.com/pthengtr/kcw-analytics.git

fatal: destination path 'kcw-analytics' already exists and is not an empty directory.


In [40]:
!cd /content/kcw-analytics && git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 2.79 KiB | 953.00 KiB/s, done.
From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
   3c171cd..a3aaf78  main       -> origin/main
Updating 3c171cd..a3aaf78
Fast-forward
 .../20_vat_sales_nonvat_purchase_report.ipynb      | 404 ++++++++-------------
 1 file changed, 145 insertions(+), 259 deletions(-)


In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")

Loaded: raw_hq_pimas_purchase_bills.csv -> (82716, 49)
Loaded: raw_hq_simas_sales_bills.csv -> (481937, 49)
Loaded: raw_hq_pidet_purchase_lines.csv -> (246580, 41)
Loaded: raw_hq_sidet_sales_lines.csv -> (1187205, 38)
Loaded: raw_hq_icmas_products.csv -> (114755, 94)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2721, 49)
Loaded: raw_syp_simas_sales_bills.csv -> (10368, 49)
Loaded: raw_syp_sidet_sales_lines.csv -> (30374, 38)
Loaded: raw_syp_pidet_purchase_lines.csv -> (25555, 41)
Loaded: raw_inventory_hq_2024.csv -> (4983, 8)


In [43]:
import sys
import importlib

# ensure repo is on path
repo_path = "/content/kcw-analytics"
if repo_path not in sys.path:
    sys.path.append(repo_path)

# import the module (NOT individual functions)
import src.kcw.utils as utils

# reload to pick up latest .py changes
importlib.reload(utils)

get_vat_sales_lines_last_purchase_nonvat = utils.get_vat_sales_lines_last_purchase_nonvat
audit_bcode_vat_sales_last_purchase = utils.audit_bcode_vat_sales_last_purchase

In [44]:
def filter_year_month(df, year, month, date_col="BILLDATE"):
    return df[pd.to_datetime(df[date_col]).dt.to_period("M") == f"{year}-{month:02d}"]

In [45]:
YEAR = 2026
MONTH = 1

vat_sales_last_purchase_nonvat_hq = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="hq"
)

vat_sales_last_purchase_nonvat_syp = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="syp"
)


In [46]:
vat_sales_last_purchase_nonvat_hq_cleaned = vat_sales_last_purchase_nonvat_hq[
    ~vat_sales_last_purchase_nonvat_hq["BILLNO"]
        .astype(str)
        .str.startswith(("TAR", "CN"), na=False)
]

vat_sales_last_purchase_nonvat_hq_cleaned = filter_year_month(vat_sales_last_purchase_nonvat_hq_cleaned, YEAR, MONTH)

vat_sales_last_purchase_nonvat_hq_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
23,1226927,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-011,10,1,12050690,...,NaN,7.0,590.0,NaN,7LPNT,N,CRRF,N,N,N
24,1226858,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-004,10,1,12050729,...,NaN,7.0,350.0,NaN,7SPNT,N,BM,N,N,N
25,1227016,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-013,10,1,12051272,...,NaN,7.0,205.0,NaN,7SPNT,N,KCY,N,N,N
27,1226983,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,20,2,12051561,...,NaN,7.0,140.0,NaN,7ICE,N,BM,N,N,N
28,1226984,1,SJ,2026-01-01 00:00:00,1.0,2026-01-01,TAD6901-012,30,3,12051563,...,NaN,7.0,90.0,NaN,7ICE,N,BM,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,1240464,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-098,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5096,1240465,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-099,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5097,1240467,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TD6901-100,10,1,30050835,...,NaN,7.0,90.0,NaN,7TET,N,CRRK,N,N,N
5099,1240501,1,SJ,2026-01-26 00:00:00,1.0,2026-01-26,TAD6901-535,20,2,30051639,...,NaN,7.0,490.0,NaN,7LAZ1,N,CRRK,N,N,N


In [47]:
vat_sales_last_purchase_nonvat_syp_cleaned = vat_sales_last_purchase_nonvat_syp[
    ~vat_sales_last_purchase_nonvat_syp["BILLNO"]
        .astype(str)
        .str.startswith(("3TAR","3CN"), na=False)
]

vat_sales_last_purchase_nonvat_syp_cleaned = filter_year_month(vat_sales_last_purchase_nonvat_syp_cleaned, YEAR, MONTH)

vat_sales_last_purchase_nonvat_syp_cleaned

,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,DED,VAT,AMOUNT,CHGAMT,ACCTNO,PAID,ACCT_NO,DONE,CANCELED,LAST_PURCHASE_ISVAT
453,1693718,1,SJ,2026-01-08 00:00:00,1,2026-01-08,3TR6901-005,30,3,13036340,...,NaN,7.0,9.0,NaN,7ชจร,N,LK,N,N,N
713,1754135,1,SJ,2026-01-13 00:00:00,1,2026-01-13,3TR6901-008,20,2,09011683,...,NaN,7.0,430.0,NaN,7อลย,N,SRI,N,N,N
878,1774163,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,20,2,05018690,...,NaN,7.0,60.0,NaN,7ชจร,N,SSWK,N,N,N
888,1774167,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,60,6,10050185,...,NaN,7.0,108.0,NaN,7ชจร,N,YOKO,N,N,N
889,1774162,1,SJ,2026-01-15 00:00:00,1,2026-01-15,3TR6901-010,10,1,12050484,...,NaN,7.0,240.0,NaN,7ชจร,N,SJP,N,N,N
1048,1774591,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,60,6,14010442,...,NaN,7.0,60.0,NaN,7ชจร,N,RJS,N,N,N
1053,1774588,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,40,4,17016070,...,NaN,7.0,160.0,NaN,7ชจร,N,LK,N,N,N
1089,1774587,1,SJ,2026-01-17 00:00:00,1,2026-01-17,3TR6901-012,30,3,33010369,...,NaN,7.0,200.0,NaN,7ชจร,N,MC,N,N,N
1123,1784711,1,SJ,2026-01-19 00:00:00,1,2026-01-19,3TR6901-013,60,6,13051791,...,NaN,7.0,28.0,NaN,7ชจร,N,SMA,N,N,N
1414,1815252,1,SJ,2026-01-23 00:00:00,1,2026-01-23,3TR6901-015,20,2,12051665,...,NaN,7.0,30.0,NaN,7ชจร,N,SMA,N,N,N


In [48]:
#sales_one, pidet_one = audit_bcode_vat_sales_last_purchase(data, "12050690", YEAR, "hq")

In [49]:
import pandas as pd
import numpy as np

def enrich_sales_with_lastcost_and_newbillno(
    data: dict,
    sales_lines: pd.DataFrame,
    *,
    source: str,  # "hq" or "syp"
    purchase_key: str = "raw_hq_pidet_purchase_lines.csv",
    # common columns
    bcode_col: str = "BCODE",
    date_col: str = "BILLDATE",
    # sales qty columns (kept for signature compatibility)
    sales_qty_col: str = "QTY",
    sales_mtp_col: str = "MTP",
    # purchase columns for cost calc
    purchase_amount_col: str = "AMOUNT",
    purchase_qty_col: str = "QTY",
    purchase_mtp_col: str = "MTP",
    # bill cols (kept for signature compatibility)
    billno_col: str = "BILLNO",
    # output
    output_cols: tuple = (
        "BCODE", "BILLDATE", "BILLNO", "DETAIL", "QTY", "MTP", "UI", "AMOUNT", "ACCT_NO",
        "CANCELED", "ISVAT", "LAST_PURCHASE_ISVAT", "LAST_COST"
    ),
) -> pd.DataFrame:
    """
    Takes already-filtered sales lines and:
      1) Looks up LAST purchase UNIT cost as-of sale date from purchases in data[purchase_key].
      2) Adds NEW_BILLNO (HQ: RV..., SYP: 3RV...)
      3) Returns requested columns (plus NEW_BILLNO).

    """

    # --- validate source ---
    source = source.lower().strip()
    if source not in ("hq", "syp"):
        raise ValueError("source must be 'hq' or 'syp'")

    bill_prefix = "RV" if source == "hq" else "3RV"

    # --- helpers ---
    def _clean_cols(df: pd.DataFrame) -> pd.DataFrame:
        out = df.copy()
        out.columns = (
            out.columns.astype(str)
            .str.replace("\ufeff", "", regex=False)
            .str.strip()
        )
        return out

    def _clean_bcode(s: pd.Series) -> pd.Series:
        return s.astype("string").str.strip()

    # --- copy & clean ---
    sales = _clean_cols(sales_lines).copy()
    purch = _clean_cols(data[purchase_key]).copy()

    # Ensure key columns exist
    for col in [bcode_col, date_col]:
        if col not in sales.columns:
            raise KeyError(f"sales_lines missing required column: {col}")
        if col not in purch.columns:
            raise KeyError(f"purchase data missing required column: {col}")

    # Clean BCODE + dates
    sales[bcode_col] = _clean_bcode(sales[bcode_col])
    purch[bcode_col] = _clean_bcode(purch[bcode_col])

    sales[date_col] = pd.to_datetime(sales[date_col], errors="coerce")
    purch[date_col] = pd.to_datetime(purch[date_col], errors="coerce")

    sales = sales.dropna(subset=[bcode_col, date_col]).copy()
    purch = purch.dropna(subset=[bcode_col, date_col]).copy()

    sales = sales[sales[bcode_col] != ""].copy()
    purch = purch[purch[bcode_col] != ""].copy()

    # Reduce purchases to only needed BCODEs (perf)
    purch = purch[purch[bcode_col].isin(sales[bcode_col].unique())].copy()

    # --- 1) LAST_COST as-of sale date (no rolling AV_COST) ---
    for c in [purchase_amount_col, purchase_qty_col, purchase_mtp_col]:
        purch[c] = pd.to_numeric(purch[c], errors="coerce")

    purch = purch.dropna(subset=[purchase_amount_col, purchase_qty_col, purchase_mtp_col]).copy()
    purch = purch[(purch[purchase_qty_col] != 0) & (purch[purchase_mtp_col] > 0)].copy()

    purch["__UNITS__"] = purch[purchase_qty_col] * purch[purchase_mtp_col]
    purch["__LASTCOST__"] = purch[purchase_amount_col] / purch["__UNITS__"].replace(0, np.nan)
    purch = purch.dropna(subset=["__LASTCOST__"]).copy()

    purch_key = purch[[bcode_col, date_col, "__LASTCOST__"]].copy()

    # merge_asof requires sorting by ON key first, then BY key
    sales_sorted = sales.sort_values([date_col, bcode_col], kind="mergesort").reset_index(drop=True)
    purch_key = purch_key.sort_values([date_col, bcode_col], kind="mergesort").reset_index(drop=True)

    merged = pd.merge_asof(
        sales_sorted,
        purch_key,
        left_on=date_col,
        right_on=date_col,
        by=bcode_col,
        direction="backward",
        allow_exact_matches=True,
    )

    merged["LAST_COST"] = merged["__LASTCOST__"]
    merged.drop(columns=["__LASTCOST__"], inplace=True)

    # --- 2) NEW_BILLNO (monthly reset, daily + 10-line chunk increment) ---
    merged["__DATEKEY__"] = merged[date_col].dt.normalize()
    merged["__MONTHKEY__"] = merged["__DATEKEY__"].dt.to_period("M")

    merged = merged.sort_values(["__DATEKEY__", bcode_col], kind="mergesort").reset_index(drop=True)

    day_chunk = merged.groupby("__DATEKEY__", sort=False).cumcount() // 20

    day_tbl = (
        pd.DataFrame({
            "__MONTHKEY__": merged["__MONTHKEY__"],
            "__DATEKEY__": merged["__DATEKEY__"],
            "day_chunk": day_chunk
        })
        .groupby(["__MONTHKEY__", "__DATEKEY__"], sort=False)["day_chunk"]
        .max()
        .rename("MAX_CHUNK")
        .reset_index()
        .sort_values(["__MONTHKEY__", "__DATEKEY__"], kind="mergesort")
    )

    day_tbl["DAY_START_SEQ0"] = (
        day_tbl.groupby("__MONTHKEY__")["MAX_CHUNK"]
        .transform(lambda x: (x + 1).cumsum() - (x + 1))
    )

    start_map = day_tbl.set_index(["__MONTHKEY__", "__DATEKEY__"])["DAY_START_SEQ0"]
    merged["__DAY_START_SEQ0__"] = start_map.loc[
        list(zip(merged["__MONTHKEY__"], merged["__DATEKEY__"]))
    ].to_numpy()

    seq = (merged["__DAY_START_SEQ0__"] + day_chunk + 1).astype(int)

    bd_yy = ((merged["__DATEKEY__"].dt.year + 543) % 100).astype(int)
    mm = merged["__DATEKEY__"].dt.month.astype(int)

    merged["NEW_BILLNO"] = (
        bill_prefix
        + bd_yy.map(lambda x: f"{x:02d}")
        + mm.map(lambda x: f"{x:02d}")
        + "-"
        + seq.map(lambda x: f"{x:03d}")
    )

    merged.drop(columns=["__DATEKEY__", "__MONTHKEY__", "__DAY_START_SEQ0__"], inplace=True)

    # --- 3) Output ---
    for col in output_cols:
        if col not in merged.columns:
            merged[col] = pd.NA

    return merged[list(output_cols) + ["NEW_BILLNO"]].copy()


In [50]:
out_syp = enrich_sales_with_lastcost_and_newbillno(
    data=data,
    sales_lines=vat_sales_last_purchase_nonvat_syp_cleaned,
    source = "syp"
)

out_hq = enrich_sales_with_lastcost_and_newbillno(
    data=data,
    sales_lines=vat_sales_last_purchase_nonvat_hq_cleaned,
    source = "hq"
)


In [51]:
out_syp

,BCODE,BILLDATE,BILLNO,DETAIL,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT,LAST_COST,NEW_BILLNO
0,13036340,2026-01-08,3TR6901-005,"แหวนสปริง 3/8"" ดำ",6.0,1.0,ตัว,9.0,LK,N,Y,N,0.24700,3RV6901-001
1,09011683,2026-01-13,3TR6901-008,ไส้กรองเครื่อง เหล็ก ก.16NF ไฮเทค3H HO7D-FM1J ...,1.0,1.0,หน่วย,430.0,SRI,N,Y,N,350.00000,3RV6901-002
2,05018690,2026-01-15,3TR6901-010,หัวแป๊บเบรค D/S620 หัวสิงห์,3.0,1.0,ตัว,60.0,SSWK,N,Y,N,15.00000,3RV6901-003
3,10050185,2026-01-15,3TR6901-010,ยางกันฝุ่นเบรค หน้า-หลัง SUMO YOKO,6.0,1.0,ตัว,108.0,YOKO,N,Y,N,9.00000,3RV6901-003
4,12050484,2026-01-15,3TR6901-010,"สกรูใบมีดดันหน้า F 5/8"" x 2"" สีทอง",12.0,1.0,หน่วย,240.0,SJP,N,Y,N,10.20000,3RV6901-003
5,14010442,2026-01-17,3TR6901-012,ลวดเชื่อมทองเหลือง เส้นเล็ก 2.6 mm.,2.0,1.0,เส้น,60.0,RJS,N,Y,N,20.16129,3RV6901-004
6,17016070,2026-01-17,3TR6901-012,แกนสกรูมิลดำ ไม่หัว M16-70m ก.1.50 ธรรมดา,8.0,1.0,ตัว,160.0,LK,N,Y,N,9.19120,3RV6901-004
7,33010369,2026-01-17,3TR6901-012,"หัวสาย+ปลอก IMS 0808 GORS 90"" MK",1.0,1.0,ชุด,200.0,MC,N,Y,N,102.00000,3RV6901-004
8,13051791,2026-01-19,3TR6901-013,หัวแป็บเบรค ตัวผู้ 1/4 หยาบ หัวสิงห์,1.0,1.0,หัว,28.0,SMA,N,Y,N,17.00000,3RV6901-005
9,12051665,2026-01-23,3TR6901-015,ตาไก่ทองเหลือง 1/4 SAK,2.0,1.0,หน่วย,30.0,SMA,N,Y,N,6.00000,3RV6901-006


In [52]:
out_hq

,BCODE,BILLDATE,BILLNO,DETAIL,QTY,MTP,UI,AMOUNT,ACCT_NO,CANCELED,ISVAT,LAST_PURCHASE_ISVAT,LAST_COST,NEW_BILLNO
0,12050690,2026-01-01,TAD6901-011,ปั้มAC/ปั้มแย็กทั้งลูก F/6610 แท้สั่ง,1.0,1.0,หน่วย,590.0,CRRF,N,Y,N,350.000000,RV6901-001
1,12050729,2026-01-01,TAD6901-004,ไส้กรองปั้มขาว F/6610 ตัวเล็ก New arrival,1.0,1.0,หน่วย,350.0,BM,N,Y,N,200.000000,RV6901-001
2,12051272,2026-01-01,TAD6901-013,เม็ดลูกปืน+หลอดยานัตถ์ พมล. F/6600 New Holland,1.0,1.0,ชุด,205.0,KCY,N,Y,N,100.000000,RV6901-001
3,12051561,2026-01-01,TAD6901-012,สกรู 6 หุน ยาว 3 นิ้ว FORD เกลียวละเอ BM,2.0,1.0,หน่วย,140.0,BM,N,Y,N,42.857095,RV6901-001
4,12051563,2026-01-01,TAD6901-012,บู๊ชใบมีดตัดหญ้า CMT รู 6 หุน BM,2.0,1.0,หน่วย,90.0,BM,N,Y,N,19.047595,RV6901-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,30050835,2026-01-26,TD6901-098,"กรองโซล่า เหล็ก L3608-L4508DI,K หัวสิงห์",1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,60.000000,RV6901-032
416,30050835,2026-01-26,TD6901-099,"กรองโซล่า เหล็ก L3608-L4508DI,K หัวสิงห์",1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,60.000000,RV6901-032
417,30050835,2026-01-26,TD6901-100,"กรองโซล่า เหล็ก L3608-L4508DI,K หัวสิงห์",1.0,1.0,หน่วย,90.0,CRRK,N,Y,N,60.000000,RV6901-032
418,30051639,2026-01-26,TAD6901-535,ยางฝาวาว KBT M9540-M108S หัวสิงห์,2.0,1.0,เส้น,490.0,CRRK,N,Y,N,60.000000,RV6901-032


In [53]:
pidet = data["raw_hq_pidet_purchase_lines.csv"].copy()
pidet_filtered = pidet[pidet["BCODE"] == "05057622"].copy()

pidet_filtered[["BCODE", 'BILLDATE', "QTY", "MTP", "PRICE", "AMOUNT"]]



,BCODE,BILLDATE,QTY,MTP,PRICE,AMOUNT


In [54]:
!apt-get -y install libpango-1.0-0 libpangoft2-1.0-0 libcairo2 libgdk-pixbuf2.0-0 libffi-dev shared-mime-info
!pip -q install weasyprint

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2 is already the newest version (1.16.0-5ubuntu2).
libffi-dev is already the newest version (3.4.2-4).
shared-mime-info is already the newest version (2.1-2).
libgdk-pixbuf2.0-0 is already the newest version (2.40.2-2build4).
libpango-1.0-0 is already the newest version (1.50.6+ds-2ubuntu1).
libpangoft2-1.0-0 is already the newest version (1.50.6+ds-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [55]:
import os
import pandas as pd
import numpy as np
from weasyprint import HTML, CSS

# -----------------------
# Thai helpers (reuse yours)
# -----------------------
TH_MONTHS_ABBR = [
    "ม.ค.", "ก.พ.", "มี.ค.", "เม.ย.", "พ.ค.", "มิ.ย.",
    "ก.ค.", "ส.ค.", "ก.ย.", "ต.ค.", "พ.ย.", "ธ.ค."
]

def thai_date(d) -> str:
    dt = pd.to_datetime(d).to_pydatetime()
    return f"{dt.day} {TH_MONTHS_ABBR[dt.month-1]} {dt.year + 543}"

def _thai_read_integer(num: int) -> str:
    if num == 0:
        return "ศูนย์"

    units = ["", "สิบ", "ร้อย", "พัน", "หมื่น", "แสน"]
    digits = ["ศูนย์", "หนึ่ง", "สอง", "สาม", "สี่", "ห้า", "หก", "เจ็ด", "แปด", "เก้า"]

    def read_under_million(n: int) -> str:
        s = ""
        d = list(map(int, str(n)))
        L = len(d)
        for i, x in enumerate(d):
            pos = L - i - 1
            if x == 0:
                continue
            if pos == 0:
                s += "เอ็ด" if (x == 1 and L > 1) else digits[x]
            elif pos == 1:
                if x == 1: s += "สิบ"
                elif x == 2: s += "ยี่สิบ"
                else: s += digits[x] + "สิบ"
            else:
                s += digits[x] + units[pos]
        return s

    out = ""
    parts = []
    while num > 0:
        parts.append(num % 1_000_000)
        num //= 1_000_000

    for i in range(len(parts)-1, -1, -1):
        n = parts[i]
        if n == 0:
            continue
        out += read_under_million(n)
        if i != 0:
            out += "ล้าน"
    return out

def thai_baht_text(amount) -> str:
    x = float(amount) if amount is not None else 0.0
    if x < 0:
        return "ลบ" + thai_baht_text(-x)

    baht = int(x)
    satang = int(round((x - baht) * 100))
    if satang == 100:
        baht += 1
        satang = 0

    baht_words = _thai_read_integer(baht) + "บาท"
    return baht_words + ("ถ้วน" if satang == 0 else _thai_read_integer(satang) + "สตางค์")

def money(x):
    try:
        return f"{float(x):,.2f}"
    except Exception:
        return ""

def esc(s):
    # minimal HTML escape
    if s is None:
        return ""
    return (str(s)
            .replace("&", "&amp;")
            .replace("<", "&lt;")
            .replace(">", "&gt;")
            .replace('"', "&quot;"))

# -----------------------
# Build ONE receipt HTML -> PDF
# -----------------------
def build_one_receipt_pdf_weasy(group_df: pd.DataFrame, pdf_path: str, *,
                               font_dir: str,
                               signature_path: str):
    # paths for @font-face (local files)
    sarabun = os.path.join(font_dir, "THSarabunNew.ttf")
    sarabun_bold = os.path.join(font_dir, "THSarabunNew-Bold.ttf")

    # header fields
    new_billno = str(group_df["NEW_BILLNO"].iloc[0])
    billdate = thai_date(group_df["BILLDATE"].iloc[0])

    branch_text = "สำนักงานใหญ่"
    if new_billno.startswith("3"):
        branch_text = "สี่แยกพัฒนา"

    # compute receipt numbers
    df = group_df.copy()
    df["QTY"] = pd.to_numeric(df.get("QTY", 0), errors="coerce").fillna(0)
    df["MTP"] = pd.to_numeric(df.get("MTP", 0), errors="coerce").fillna(0)
    last_cost = pd.to_numeric(df.get("LAST_COST", np.nan), errors="coerce")

    df["UNIT_PRICE"] = last_cost
    df["AMOUNT_CALC"] = last_cost.fillna(0) * df["QTY"] * df["MTP"]

    grand_total = float(df["AMOUNT_CALC"].sum())
    thai_words = thai_baht_text(grand_total)

    # build table rows
    rows_html = []
    for _, r in df.iterrows():
        bcode = esc(r.get("BCODE", ""))
        detail = esc(r.get("DETAIL", "")).replace("\n", " ")
        unit_price = money(r["UNIT_PRICE"]) if pd.notna(r["UNIT_PRICE"]) else "UNKNOWN"
        qty = r.get("QTY", 0)
        qty_str = money(qty) if (float(qty) % 1 != 0) else str(int(qty))
        unit = esc(r.get("UI", ""))
        total = money(r.get("AMOUNT_CALC", 0))

        rows_html.append(f"""
          <tr>
            <td class="c-bcode">{bcode}</td>
            <td class="c-detail">{detail}</td>
            <td class="c-num">{unit_price}</td>
            <td class="c-num">{qty_str}</td>
            <td class="c-unit">{unit}</td>
            <td class="c-num">{total}</td>
          </tr>
        """)

    rows_html = "\n".join(rows_html)

    # signature path: WeasyPrint likes file:// URLs
    sig_url = "file://" + os.path.abspath(signature_path)

    html = f"""
<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <style>
    @font-face {{
      font-family: "Sarabun";
      src: url("file://{os.path.abspath(sarabun)}");
    }}
    @font-face {{
      font-family: "Sarabun";
      src: url("file://{os.path.abspath(sarabun_bold)}");
      font-weight: bold;
    }}

    @page {{
      size: A4;
      margin: 18px 24px;
    }}

    body {{
      font-family: "Sarabun";
      font-size: 14px;
      line-height: 1.35;
      color: #000;
    }}

    .title {{
      font-size: 22px;
      font-weight: bold;
      margin-bottom: 8px;
    }}

    .hdr {{
      display: flex;
      justify-content: flex-end;
      gap: 24px;
      margin-bottom: 10px;
    }}
    .hdr-col {{
      text-align: right;
    }}
    .lbl {{
      font-weight: bold;
    }}

    .block {{
      margin: 6px 0;
    }}

    table {{
      width: 100%;
      border-collapse: collapse;
      margin-top: 8px;
    }}
    th, td {{
      border: 1px solid #000;
      padding: 4px 6px;
      vertical-align: top;
    }}
    th {{
      background: #f2f2f2;
      text-align: center;
      font-weight: bold;
    }}

    .c-bcode {{ width: 14%; }}
    .c-detail {{ width: 46%; }}
    .c-unit  {{ width: 8%; text-align: center; }}
    .c-num   {{ width: 10%; text-align: right; white-space: nowrap; }}

    .totals {{
      margin-top: 10px;
      text-align: right;
      font-weight: bold;
      font-size: 16px;
    }}
    .words {{
      margin-top: 2px;
      text-align: right;
      font-size: 13px;
      font-weight: normal;
    }}

    .sign-wrap {{
      margin-top: 18px;
      display: flex;
      justify-content: flex-end;
    }}
    .sign-table {{
      border-collapse: collapse;
      width: 320px;
    }}
    .sign-table td {{
      border: none;
      padding: 2px 6px;
    }}
    .sign-img {{
      width: 188px;
      height: 48px;
      object-fit: contain;
      display: block;
    }}
    .note {{
      margin-top: 10px;
      text-align: right;
      font-size: 12px;
    }}
  </style>
</head>

<body>


  <div class="hdr">
    <div class="hdr-col">
      <div class="title">ใบสำคัญรับเงิน</div>
      <div><span class="lbl">เลขที่:</span> {esc(new_billno)}</div>
      <div><span class="lbl">วันที่:</span> {esc(billdate)}</div>
    </div>
  </div>

  <div class="block"><span class="lbl">ข้าพเจ้า:</span> นางสาวนฤมล วิทยผโลทัย (ผู้ขายสินค้า)</div>
  <div class="block"><span class="lbl">ที่อยู่:</span> 305 หมู่ 1 ตำบล ชุมแสง อำเภอ วังจันทร์ จังหวัด ระยอง</div>
  <div class="block"><span class="lbl">เลขประจำตัวผู้เสียภาษี:</span> 1-2001-99001-42-8</div>

  <div class="block" style="margin-top:10px;">
    ได้รับเงินจาก บริษัทเกียรติชัยอะไหล่ยนต์ 2007 จำกัด ({esc(branch_text)}) (ผู้ซื้อ) ดังรายการต่อไปนี้
  </div>

  <table>
    <thead>
      <tr>
        <th>รหัสสินค้า</th>
        <th>รายการ</th>
        <th>ราคา/หน่วย</th>
        <th>จำนวน</th>
        <th>หน่วย</th>
        <th>รวมยอดเงิน</th>
      </tr>
    </thead>
    <tbody>
      {rows_html}
    </tbody>
  </table>

  <div class="totals">รวมทั้งสิ้น: {money(grand_total)}</div>
  <div class="words">จำนวนเงิน (ตัวอักษร): {esc(thai_words)}</div>

  <div class="sign-wrap">
    <table class="sign-table">
      <tr>
        <td style="text-align:right; width:110px;">ผู้รับเงิน</td>
        <td style="text-align:right;"><img class="sign-img" src="{sig_url}"></td>
      </tr>
      <tr>
        <td style="text-align:right;">ผู้จ่ายเงิน</td>
        <td style="text-align:right;"><img class="sign-img" src="{sig_url}"></td>
      </tr>
    </table>
  </div>

  <div class="note">หมายเหตุ: แนบสำเนาบัตรประชาชนผู้รับเงิน</div>
</body>
</html>
"""

    HTML(string=html).write_pdf(pdf_path)


def build_receipts_by_new_billno_weasy(df: pd.DataFrame, out_dir: str, *,
                                      font_dir: str,
                                      signature_path: str):
    os.makedirs(out_dir, exist_ok=True)
    for new_billno, g in df.groupby("NEW_BILLNO", sort=True):
        pdf_path = os.path.join(out_dir, f"{new_billno}.pdf")
        build_one_receipt_pdf_weasy(g, pdf_path, font_dir=font_dir, signature_path=signature_path)
    return out_dir


In [56]:
import logging

# WeasyPrint
logging.getLogger("weasyprint").setLevel(logging.ERROR)
logging.getLogger("weasyprint.progress").setLevel(logging.ERROR)
logging.getLogger("weasyprint.CSS").setLevel(logging.ERROR)
logging.getLogger("weasyprint.HTML").setLevel(logging.ERROR)

# fontTools (the spam you're seeing)
logging.getLogger("fontTools").setLevel(logging.ERROR)
logging.getLogger("fontTools.subset").setLevel(logging.ERROR)
logging.getLogger("fontTools.ttLib").setLevel(logging.ERROR)

# Optional: also silence warnings from fontTools tables
logging.getLogger("fontTools.ttLib.tables").setLevel(logging.ERROR)

In [57]:
out_dir = build_receipts_by_new_billno_weasy(
    df=out_syp,   # or your filtered month df
    out_dir=f"/content/drive/MyDrive/kcw_analytics/04_outputs/[3RV]vat_sales_nonvat_purchase_receipts_syp_{YEAR}_{MONTH}",
    font_dir="/content/drive/MyDrive/kcw_analytics/00_fonts/THSarabunNew",
    signature_path="/content/drive/MyDrive/kcw_analytics/00_fonts/Signature.jpg"
)

print("Saved to:", out_dir)

Saved to: /content/drive/MyDrive/kcw_analytics/04_outputs/[3RV]vat_sales_nonvat_purchase_receipts_syp_2026_1


In [58]:
out_dir = build_receipts_by_new_billno_weasy(
    df=out_hq,   # or your filtered month df
    out_dir=f"/content/drive/MyDrive/kcw_analytics/04_outputs/[RV]vat_sales_nonvat_purchase_receipts_hq_{YEAR}_{MONTH}",
    font_dir="/content/drive/MyDrive/kcw_analytics/00_fonts/THSarabunNew",
    signature_path="/content/drive/MyDrive/kcw_analytics/00_fonts/Signature.jpg"
)

print("Saved to:", out_dir)


Saved to: /content/drive/MyDrive/kcw_analytics/04_outputs/[RV]vat_sales_nonvat_purchase_receipts_hq_2026_1


In [59]:
out_hq.to_csv(f"/content/drive/MyDrive/kcw_analytics/04_outputs/[RV]vat_sales_nonvat_purchase_hq_{YEAR}_{MONTH}.csv", index=False, encoding="utf-8-sig")
out_syp.to_csv(f"/content/drive/MyDrive/kcw_analytics/04_outputs/[3RV]vat_sales_nonvat_purchase_syp_{YEAR}_{MONTH}.csv", index=False, encoding="utf-8-sig")